In [1]:
import sqlite3

In [2]:
table_names = ['counts_train', 'counts_val', 'counts_test', 'bbx_train', 'bbx_val']

counts_table = ''' (
    id integer not null primary key autoincrement,
    image_path text,
    category integer,
    box_count integer)'''

bbx_table = ''' (
    id integer not null primary key autoincrement,
    counts_id integer,
    image_path text,
    category integer,
    x integer,
    y integer,
    width integer,
    height integer,
    blur integer,
    expression integer,
    illumination integer,
    invalid integer,
    occlusion integer,
    pose integer)'''

counts_train = f'counts_train {counts_table}'
counts_val = f'counts_val {counts_table}'
counts_test = f'counts_test {counts_table}'
bbx_train = f'bbx_train {bbx_table}'
bbx_val = f'bbx_val {bbx_table}'

db_tables = [ counts_train, counts_val, counts_test, bbx_train, bbx_val ]

In [3]:
def create_database(name = 'widerface.db'):
    con = sqlite3.connect(name)
    if not con:
        raise RuntimeError(f'Unable to load the database: "{name}"')
    cur = con.cursor()
    for tb in db_tables:
        cur.execute(f'create table if not exists {tb}')
    con.close()
    
def drop_tables(name = 'widerface.db'):
    con = sqlite3.connect(name)
    cur = con.cursor()
    for tb in table_names:
        cur.execute(f'drop table if exists {tb}')
    con.close()

In [4]:
drop_tables()
create_database()

In [5]:
def add_bbx_data(bbx_table, counts_table, file_path, db_name = 'widerface.db'):

    con = sqlite3.connect(db_name)
    
    def insert_bbx_entry(values):
        try:
            cur = con.cursor()
            cur.execute(f'''insert into {bbx_table} (counts_id, image_path, category, x, y, width, 
                height, blur, expression, illumination, invalid, occlusion, pose)
                values(?,?,?,?,?,?,?,?,?,?,?,?,?)''', values)
        except sqlite3.IntegrityError as detail:
            print(detail)
            pass
        
    def insert_counts_entry(values):
        try:
            cur = con.cursor()
            cur.execute(f'insert into {counts_table} (image_path, category, box_count) values(?,?,?)', values)
        except sqlite3.IntegrityError as detail:
            print(detail)
            pass
        id = cur.lastrowid
        return id


    with open(file_path ,'r') as fp:
        last_image = ''
        get_count = False
        face_count = 0
        counts_id = 0
        category = 0
        for i, line in enumerate(fp):
            if line.endswith('.jpg\n'):
                last_image = line[0:-1]
                category = int(last_image.split('--')[0])
                get_count = True
            elif get_count:
                image_counts = (last_image, category, int(line))
                counts_id = insert_counts_entry(image_counts)
                get_count = False
            else:
                data_str = line.rstrip().split(' ')
                data = [int(x) for x in data_str ]
                bbx_entries = tuple([counts_id, last_image, category] + data)
                insert_bbx_entry(bbx_entries)
                
        con.commit()

    con.close()

def add_test_images(file_path, db_name = 'widerface.db'):
    
    con = sqlite3.connect(db_name)
    
    def insert_entry(values):
        try:
            cur = con.cursor()
            cur.execute(f'insert into counts_test (image_path, category, box_count) values(?,?,?)', values)
        except sqlite3.IntegrityError as detail:
            print(detail)
            pass
        id = cur.lastrowid
        return id
    
    with open(file_path ,'r') as fp:
        for i, line in enumerate(fp):
            values = (line[0:-1], -1, 0)
            insert_entry(values)
        con.commit()
            
    con.close()

## Run this cell to populate the training set data. Only run once.

In [6]:
add_bbx_data('bbx_train','counts_train','./data/wider_face_split/wider_face_train_bbx_gt.txt')

## Run this cell to populate the validation set data. Only run once.

In [7]:
add_bbx_data('bbx_val','counts_val','./data/wider_face_split/wider_face_val_bbx_gt.txt')

## Run this cell to populate the test file paths. Only run once.

In [8]:
add_test_images('./data/wider_face_split/wider_face_test_filelist.txt')